# Base model

This is notebook for checking inference and knowledge of base model.

*Base model*: [deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B](https://huggingface.co/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B) - the smallest reasoning DeepSeek-R1 model.

### Check base LLM inference

In [1]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [2]:
from unsloth import FastLanguageModel

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=4096,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


m:\python_projects\MedAlign-LLM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-10 15:09:55 [__init__.py:256] Automatically detected platform cuda.


m:\python_projects\MedAlign-LLM\venv\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.0. vLLM: 0.8.0.
   \\   /|    NVIDIA GeForce RTX 4060 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
from transformers import TextStreamer
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
)
model = FastLanguageModel.for_inference(model)

In [4]:
prompt = """
You are an expert in solving multiple-choice questions accurately and explaining your reasoning clearly.
Given a question and a list of answer choices (A, B, C, D), your task is to:
1. Reason shortly about the question and answer choices to find evidances to support your answer.
2. Identify the correct answer.
3. Output the final answer in the format: Answer: [Option Letter]

Here is a question: Which vitamin is supplied from only animal source?
A. Vitamin C
B. Vitamin B7
C. Vitamin B12
D. Vitamin D

Reasoning:
"""
inputs = tokenizer([prompt], return_tensors="pt", padding=True, truncation=True).to("cuda")

text_streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
_ = model.generate(**inputs, streamer=text_streamer, max_new_tokens=4096)

Okay, I need to figure out which vitamin is only provided from animals. I remember that vitamins are usually found in different sources: some are plant-based, others are animal-based.

Vitamin C is a carotenoid found in tomatoes, carrots, and spinach. So that's a plant-based source.

Vitamin B12 is also known as riboflavin, and it's found in leafy greens like spinach, kale, and also in some fish like salmon. So again, plant-based.

Vitamin B7 is something I'm a bit fuzzy on. Wait, I think B7 is something related to bone health. Oh right, it's called thiamine, and it's found in animal feed, specifically in red meat, poultry, and beans. So that's an animal source.

Vitamin D is important for bone health. It's found in fish, shellfish, and some fruits. So that's also an animal source.

So putting it together, the only vitamin provided from an animal source would be B7, which is thiamine. So the answer should be option B.
</think>

The correct answer is B. Vitamin B7 (thiamine) is supplied

Correct answer is Vitamin B12.

Base model gave wrong answer.